In [11]:
# imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [13]:
# years to be used
years = list(range(2023, 2021, -1))

In [15]:
all_matches = []

In [17]:
# stats url
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [25]:
# for each year in years
for year in years:
    ## get data on the page
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    ## select the stats table
    standings_table = soup.select('table.stats_table')[0]
    
    ## find all the links in the table
    # find all links
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    # get links for squads
    links = [l for l in links if '/squads/' in l]
    # create absolute links
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # prev season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    ## for each team in the table
    for team_url in team_urls:
        # get only team name from the link
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        
        ## get data inside the team url and save all the fixture data
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        ## get shooting stats
        # init soup
        soup = BeautifulSoup(data.text) 
        # find all links
        links = soup.find_all('a') 
        # get all the links on page
        links = [l.get("href") for l in links] 
        # all links with shooting
        links = [l for l in links if l and 'all_comps/shooting/' in l] 
        # get data from first link from array created above (all links are the same)
        data = requests.get(f"https://fbref.com{links[0]}")  
        # read in shooting stats with pandas
        shooting = pd.read_html(data.text, match = "Shooting")[0]
        # drop level because of shape
        shooting.columns = shooting.columns.droplevel() 
        
        # try lop as some teams don't have shooting stats
        try:
            # merge tables
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            # ignore and continue
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(20) # slow down so not getting blocked
        
            

1


IndexError: list index out of range

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:

match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")